## Execute the steps manually in the notebook ##

### Fill table below to submit batch job

In [1]:
user_data = {
    "username": "test_user",
    "password": "test_password",
    "Client_ID": "rick sorkin",
    "User_Project_ID": "sampleproject-440900",
    "User_Dataset_ID": "user_dataset",
    "Input_Table_ID": "input_table",
    "Output_Table_ID": "output_2",
    "Model": "gpt-3.5-turbo",
    "API_key": ""  
}

## Script that will submit job

In [3]:
import logging
import json
from jobPrep import retrieve_api_key, get_job_data, submit_job
from jobStatus import wait_for_completion, check_job_status

# Step 1: Retrieve API Key
logging.info("🔑 Retrieving API key...")
api_key = retrieve_api_key(user_data)

# Step 2: Prepare and Submit Job
if api_key:
    logging.info("📋 Preparing job data...")
    job_data = get_job_data(user_data)
    logging.info("📤 Submitting job...")
    job_id = submit_job(api_key, job_data)

    # Step 3: Monitor Job Status
    if job_id:
        logging.info("⏳ Monitoring job status...")
        final_status = wait_for_completion(job_id, user_data["Client_ID"], api_key,3)
        if final_status:
            logging.info("📊 Final Job Status Retrieved:")
            logging.info(json.dumps(final_status, indent=4))

2024-11-21 19:36:44,028 - INFO - 🔑 Retrieving API key...
2024-11-21 19:36:44,109 - INFO - API key retrieved successfully: c1ab6a79-a7a8-4fd9-945e-08678849debd
2024-11-21 19:36:44,110 - INFO - 📋 Preparing job data...
2024-11-21 19:36:44,110 - INFO - 📤 Submitting job...
2024-11-21 19:36:44,237 - INFO - Job submitted successfully.
2024-11-21 19:36:44,238 - INFO - ⏳ Monitoring job status...
2024-11-21 19:36:44,239 - INFO - 🔎 Checking for Job Status...
2024-11-21 19:36:44,344 - ERROR - Failed to retrieve job status: 500, {"details":"'NoneType' object has no attribute 'get'","error":"Failed to retrieve job status"}

2024-11-21 19:36:44,345 - ERROR - ❌ Could not retrieve job status. Retrying...
2024-11-21 19:36:47,347 - INFO - 🔎 Checking for Job Status...
2024-11-21 19:36:47,453 - INFO - Job beb83b57-848a-410a-a0f6-07d3ed259ac3 in progress: 3 / 13
2024-11-21 19:36:47,454 - INFO - ⏳ Job is still in progress. Retrying in a few seconds...
2024-11-21 19:36:50,455 - INFO - 🔎 Checking for Job Statu

### Manual Job Status Check

In [4]:
check_job_status(job_id=job_id, client_id=user_data["Client_ID"], api_key=api_key)

2024-11-21 19:37:05,240 - INFO - 🔎 Checking for Job Status...
2024-11-21 19:37:05,347 - INFO - Job beb83b57-848a-410a-a0f6-07d3ed259ac3 is complete. Processed 13 rows.


{'status': 'complete',
 'details': {'Client_ID': 'rick sorkin',
  'Job_ID': 'beb83b57-848a-410a-a0f6-07d3ed259ac3',
  'current_row': 13,
  'total_rows': 13}}